In [1]:
!pip install tigramite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.6/309.6 kB 7.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from tigramite import plotting as tp
import matplotlib.pyplot as plt
from pathlib import Path

In [4]:
def get_nonlinear_equations(n_vars, max_lag):
    """Get nonlinear polynomial equations for specified configuration"""
    if n_vars == 4:
        if max_lag == 2:
            return [
                "X4[t] = 0.25 * X1[t-2]^2 - 0.1 * X1[t-2]^3 + e4",
                "X3[t] = 0.35 * X4[t]^2 - 0.15 * X4[t]^3 + e3",
                "X2[t] = 0.3 * X3[t-1]^2 - 0.05 * X3[t-1]^3 + e2",
                "X1[t] = 0.4 * X2[t]^2 - 0.2 * X2[t]^3 + e1"
            ]
        elif max_lag == 3:
            return [
                "X4[t] = 0.25 * X1[t-2]^2 - 0.1 * X1[t-2]^3 + e4",
                "X3[t] = 0.35 * X4[t]^2 - 0.15 * X4[t]^3 + 0.2 * X2[t-3]^2 + e3",
                "X2[t] = 0.3 * X3[t-1]^2 - 0.05 * X3[t-1]^3 + e2",
                "X1[t] = 0.4 * X2[t]^2 - 0.2 * X2[t]^3 + e1"
            ]
        elif max_lag == 4:
            return [
                "X4[t] = 0.25 * X1[t-4]^2 - 0.1 * X1[t-4]^3 + e4",
                "X3[t] = 0.35 * X4[t]^2 - 0.15 * X4[t]^3 + 0.2 * X2[t-3]^2 + e3",
                "X2[t] = 0.3 * X3[t-1]^2 - 0.05 * X3[t-1]^3 + e2",
                "X1[t] = 0.4 * X2[t]^2 - 0.2 * X2[t]^3 + e1"
            ]
    elif n_vars == 6:
        base_equations = get_nonlinear_equations(4, max_lag)
        additional = [
            "X6[t] = 0.45 * X5[t]^2 - 0.15 * X5[t]^3 + e6",
            "X5[t] = 0.3 * X4[t-1]^2 - 0.1 * X4[t-1]^3 + e5"
        ]
        return additional + base_equations
    elif n_vars == 8:
        base_equations = get_nonlinear_equations(6, max_lag)
        additional = [
            "X8[t] = 0.4 * X7[t]^2 - 0.12 * X7[t]^3 + e8",
            "X7[t] = 0.35 * X6[t-1]^2 - 0.08 * X6[t-1]^3 + e7"
        ]
        return additional + base_equations
    return []

class BlockMissingNonlinearGenerator:
    def __init__(self, block_size_range=(5, 20), block_prob=0.2, noise_type='gaussian',
                 noise_params={'scale': 0.1, 'df': 3}, random_state=None):
        """
        Initialize generator with block missingness pattern

        Parameters:
        block_size_range: tuple, (min_size, max_size) for missing blocks
        block_prob: float between 0 and 1, probability of starting a missing block
        noise_type: str, 'gaussian' or 'student_t'
        noise_params: dict with 'scale' and 'df' (for Student's t) parameters
        random_state: int, random seed
        """
        self.block_size_range = block_size_range
        self.block_prob = block_prob
        self.noise_type = noise_type
        self.noise_params = noise_params
        self.random_state = random_state
        if random_state is not None:
            np.random.seed(random_state)

    def generate_noise(self, size):
        """Generate noise based on specified distribution"""
        if self.noise_type == 'gaussian':
            return np.random.normal(0, self.noise_params['scale'], size=size)
        elif self.noise_type == 'student_t':
            return stats.t.rvs(df=self.noise_params['df'],
                             loc=0,
                             scale=self.noise_params['scale'],
                             size=size)

    def generate_irregular_timestamps(self, n_points, total_time, min_gap=0.1):
        """Generate irregular sampling times"""
        times = np.zeros(n_points)
        times[0] = np.random.uniform(0, min_gap)

        for i in range(1, n_points):
            gap = np.random.exponential(scale=(total_time-times[i-1])/(n_points-i))
            times[i] = times[i-1] + max(gap, min_gap)

            if times[i] > total_time:
                times = times * (total_time / times[i])

        return times

    def find_nearest_lag_idx(self, timestamps, current_idx, lag_time):
        """Find index of nearest available past observation for given lag"""
        target_time = timestamps[current_idx] - lag_time
        past_timestamps = timestamps[:current_idx]
        if len(past_timestamps) == 0:
            return 0
        return (np.abs(past_timestamps - target_time)).argmin()

    def generate_block_missing_pattern(self, n_points, n_vars):
        """Generate block missingness pattern (MAR/NMAR)"""
        missing_mask = np.zeros((n_points, n_vars), dtype=bool)
        current_block = False
        block_remaining = 0

        for t in range(n_points):
            if not current_block and np.random.random() < self.block_prob:
                # Start new missing block
                current_block = True
                block_remaining = np.random.randint(*self.block_size_range)

                # Choose which variables to affect (MAR - depends on other variables)
                n_affected = np.random.randint(1, n_vars + 1)
                affected_vars = np.random.choice(n_vars, size=n_affected, replace=False)

                # Set missing pattern for this block
                for _ in range(block_remaining):
                    if t + _ < n_points:
                        missing_mask[t + _, affected_vars] = True

            if current_block:
                block_remaining -= 1
                if block_remaining <= 0:
                    current_block = False

        return missing_mask

    def generate_equations(self, t, X, lag_indices, n_vars, max_lag):
        """Execute nonlinear polynomial equations with irregular sampling"""
        noise = self.generate_noise(n_vars)
        equations = get_nonlinear_equations(n_vars, max_lag)
        true_links = extract_coefficients_from_equations(equations)

        # Generate values based on nonlinear relationships
        for i in range(n_vars-1, -1, -1):
            var_name = f'X{i+1}'
            value = 0

            # Add causal influences with polynomial terms
            for (source, lag, target, power), coef in true_links.items():
                if target == var_name:
                    source_idx = int(source[1:]) - 1
                    if lag == 0:
                        value += coef * (X[t, source_idx] ** power)
                    else:
                        lag_idx = lag_indices[abs(lag)-1]
                        value += coef * (X[lag_idx, source_idx] ** power)

            # Add noise term
            X[t, i] = value + noise[i]

    def generate_multivariate_ts(self, n_points, n_vars, max_lag, total_time=100, min_gap=0.1):
        """Generate multivariate time series with irregular sampling and block missingness"""
        # Initialize arrays
        X = np.zeros((n_points, n_vars))

        # Generate irregular timestamps
        timestamps = self.generate_irregular_timestamps(n_points, total_time, min_gap)

        # Initialize first steps with noise
        for i in range(max_lag):
            X[i] = self.generate_noise(n_vars)

        # Generate time series
        for t in range(max_lag, n_points):
            mean_diff = np.mean(np.diff(timestamps))
            lag_indices = [self.find_nearest_lag_idx(timestamps, t, i * mean_diff)
                         for i in range(1, max_lag + 1)]

            self.generate_equations(t, X, lag_indices, n_vars, max_lag)

        # Generate and apply block missingness pattern
        missing_mask = self.generate_block_missing_pattern(n_points, n_vars)
        X_missing = X.copy()
        X_missing[missing_mask] = np.nan

        # Create DataFrames for both complete and missing data
        columns = [f'X{i+1}' for i in range(n_vars)]
        df_missing = pd.DataFrame(X_missing, columns=columns)
        df_missing['time'] = timestamps

        df_complete = pd.DataFrame(X, columns=columns)
        df_complete['time'] = timestamps

        return df_missing, df_complete, missing_mask

def extract_coefficients_from_equations(equations):
    """Extract coefficients and causal relationships from nonlinear equations"""
    causal_links = {}

    for eq in equations:
        if '=' not in eq:
            continue

        left, right = [side.strip() for side in eq.split('=')]
        target = left.split('[')[0]

        terms = [term.strip() for term in right.split('+')]
        for term in terms:
            if '*' in term and 'X' in term:
                parts = term.split('*')
                coeff = float(parts[0].strip())
                var_part = parts[1].split('^')[0].strip()
                var = var_part.split('[')[0]
                lag_part = var_part.split('[')[1].split(']')[0]

                power = 1
                if '^' in term:
                    power = int(term.split('^')[1].split(' ')[0])

                lag = 0 if lag_part == 't' else -int(lag_part.split('-')[1])
                causal_links[(var, lag, target, power)] = coeff

    return causal_links

def extract_linear_links_for_graph(equations):
    """Extract base linear coefficients for graph visualization"""
    links = {}

    for eq in equations:
        if '=' in eq:
            left, right = [side.strip() for side in eq.split('=')]
            target = left.split('[')[0]

            terms = [term.strip() for term in right.split('+')]
            for term in terms:
                if '*' in term and 'X' in term:
                    # Get base coefficient (first number from polynomial terms)
                    parts = term.split('*')
                    coeff = float(parts[0].strip())
                    var_part = parts[1].strip()

                    # Get variable name without power terms
                    var = var_part.split('^')[0].split('[')[0]
                    lag_part = var_part.split('[')[1].split(']')[0]

                    lag = 0 if lag_part == 't' else -int(lag_part.split('-')[1])
                    links[(var, lag, target)] = coeff

    return links

def save_dataset_and_visualizations(df_missing, df_complete, missing_mask, n_vars, max_lag,
                                  sample_size, noise_type, block_prob, output_dir="output_block_missing"):
    """Save dataset and create visualizations including causal graph"""
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    base_filename = f'{output_dir}/nonlinear_ts_n{sample_size}_vars{n_vars}_lag{max_lag}_{noise_type}_block{int(block_prob*100)}'

    # Save datasets
    df_missing.to_csv(f'{base_filename}_missing.csv', index=False)
    df_complete.to_csv(f'{base_filename}_complete.csv', index=False)

    # Create and save causal graph
    equations = get_nonlinear_equations(n_vars, max_lag)
    true_links = extract_linear_links_for_graph(equations)

    # Create matrices for tigramite plotting
    var_names = [f'X{i+1}' for i in range(n_vars)]
    val_matrix = np.zeros((n_vars, n_vars, max_lag + 1))
    graph_matrix = np.zeros((n_vars, n_vars, max_lag + 1), dtype='bool')

    # Fill matrices based on true links
    for (source, lag, target), weight in true_links.items():
        source_idx = int(source[1:]) - 1
        target_idx = int(target[1:]) - 1
        lag_idx = abs(lag)

        # Add the link to the matrices
        val_matrix[source_idx, target_idx, lag_idx] = weight
        graph_matrix[source_idx, target_idx, lag_idx] = True

        # For contemporaneous links, make val_matrix symmetric
        if lag == 0:
            val_matrix[target_idx, source_idx, lag_idx] = weight

    # Plot and save causal graph
    plt.figure(figsize=(12, 12))
    tp.plot_time_series_graph(
        val_matrix=val_matrix,
        graph=graph_matrix,
        var_names=var_names,
        link_colorbar_label='Nonlinear Effect Strength',
        node_size=0.05
    )
    plt.title(f'Nonlinear Causal Graph\n(n={sample_size}, vars={n_vars}, lag={max_lag})\n{noise_type}, Block Prob {block_prob:.1f}')
    plt.savefig(f'{base_filename}_causal_graph.png')
    plt.close()

    # Save causal structure description
    with open(f'{base_filename}_causal_structure.txt', 'w') as f:
        f.write(f"True Nonlinear Causal Structure:\n")
        f.write("Format: (source, lag, target) => coefficient\n")
        f.write("\nEquations:\n")
        for eq in equations:
            f.write(f"{eq}\n")
        f.write("\nNonlinear Causal Links:\n")
        for (source, lag, target), coef in true_links.items():
            f.write(f"({source}, {lag}, {target}) => {coef} [base coefficient]\n")

    # Plot time series with missing blocks
    plt.figure(figsize=(15, 10))
    for col in df_missing.columns[:-1]:  # Exclude time column
        plt.plot(df_complete['time'], df_complete[col], label=f'{col} (complete)', alpha=0.3)
        mask = ~df_missing[col].isna()
        plt.scatter(df_missing.loc[mask, 'time'], df_missing.loc[mask, col],
                   label=f'{col} (observed)', alpha=0.7, s=20)

    plt.title(f'Nonlinear Time Series with Block Missingness\n(n={sample_size}, vars={n_vars}, lag={max_lag})')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{base_filename}_series.png')
    plt.close()

    # Plot missing data pattern
    plt.figure(figsize=(15, 5))
    plt.imshow(missing_mask.T, aspect='auto', cmap='binary')
    plt.title('Missing Data Pattern (black = missing)')
    plt.xlabel('Time')
    plt.ylabel('Variable')
    plt.yticks(range(n_vars), [f'X{i+1}' for i in range(n_vars)])
    plt.colorbar(label='Missing')
    plt.savefig(f'{base_filename}_missing_pattern.png')
    plt.close()

    # Save missing data statistics
    with open(f'{base_filename}_statistics.txt', 'w') as f:
        f.write(f"Missing Data Statistics:\n")
        f.write(f"Total points: {missing_mask.size}\n")
        f.write(f"Missing points: {missing_mask.sum()}\n")
        f.write(f"Missing percentage: {100 * missing_mask.sum() / missing_mask.size:.2f}%\n")
        f.write("\nMissing percentage by variable:\n")
        for i in range(n_vars):
            pct = 100 * missing_mask[:, i].sum() / len(missing_mask)
            f.write(f"X{i+1}: {pct:.2f}%\n")

def analyze_block_missing_data(df_missing, df_complete, missing_mask, title="Nonlinear Time Series with Block Missingness"):
    """Analyze and visualize block missing data patterns"""
    print(f"\nAnalyzing {title}")
    print("=" * 50)

    # Basic missing data statistics
    n_total = missing_mask.size
    n_missing = missing_mask.sum()
    missing_rate = n_missing / n_total

    print(f"\nMissing Data Summary:")
    print(f"Total values: {n_total}")
    print(f"Missing values: {n_missing}")
    print(f"Overall missing rate: {missing_rate:.2%}")

    # Analyze block characteristics
    block_lengths = []
    current_block = 0

    for col in range(missing_mask.shape[1]):
        block_start = False
        for row in range(missing_mask.shape[0]):
            if missing_mask[row, col]:
                if not block_start:
                    block_start = True
                    current_block = 1
                else:
                    current_block += 1
            elif block_start:
                block_lengths.append(current_block)
                block_start = False
                current_block = 0
        if block_start:  # Handle case where block ends at the end of series
            block_lengths.append(current_block)

    if block_lengths:
        print("\nBlock Length Statistics:")
        print(f"Number of blocks: {len(block_lengths)}")
        print(f"Mean block length: {np.mean(block_lengths):.2f}")
        print(f"Median block length: {np.median(block_lengths):.2f}")
        print(f"Min block length: {min(block_lengths)}")
        print(f"Max block length: {max(block_lengths)}")

    # Statistical comparison between complete and observed data
    print("\nStatistical Comparison (Complete vs. Observed):")
    for col in df_missing.columns[:-1]:  # Exclude time
        complete_stats = df_complete[col]
        observed_stats = df_missing[col].dropna()

        print(f"\n{col}:")
        print(f"Complete  - Mean: {np.mean(complete_stats):.3f}, Std: {np.std(complete_stats):.3f}")
        print(f"Observed - Mean: {np.mean(observed_stats):.3f}, Std: {np.std(observed_stats):.3f}")

    return block_lengths

def generate_all_combinations():
    """Generate datasets for all combinations of parameters"""

if __name__ == "__main__":
    print("Generating nonlinear time series with block missingness...")

    # # Example case for testing
    # n_points = 1000
    # n_vars = 4
    # max_lag = 2
    # block_prob = 0.2

    # # Generate example with Gaussian noise
    # generator = BlockMissingNonlinearGenerator(
    #     block_size_range=(5, 20),
    #     block_prob=block_prob,
    #     noise_type='gaussian',
    #     noise_params={'scale': 0.1},
    #     random_state=42
    # )

    # # Generate dataset
    # df_missing, df_complete, missing_mask = generator.generate_multivariate_ts(
    #     n_points=n_points,
    #     n_vars=n_vars,
    #     max_lag=max_lag,
    #     total_time=100,
    #     min_gap=0.1
    # )

    # # Save and analyze example case
    # save_dataset_and_visualizations(
    #     df_missing=df_missing,
    #     df_complete=df_complete,
    #     missing_mask=missing_mask,
    #     n_vars=n_vars,
    #     max_lag=max_lag,
    #     sample_size=n_points,
    #     noise_type='gaussian',
    #     block_prob=block_prob
    # )

    # # Analyze the data
    # block_lengths = analyze_block_missing_data(
    #     df_missing,
    #     df_complete,
    #     missing_mask,
    #     "Gaussian Example with Block Missingness"
    # )

    # # Generate all combinations
    # Uncomment the following line to generate full dataset
    # generate_all_combinations()
    sample_sizes = [500, 1000, 3000, 5000]
    n_vars_list = [4, 6, 8]
    max_lags = [2, 3, 4]
    noise_types = ['gaussian', 'student_t']
    block_probs = [0.1, 0.2, 0.3]

    for n in sample_sizes:
        for vars in n_vars_list:
            for lag in max_lags:
                for noise_type in noise_types:
                    for prob in block_probs:
                        print(f"\nGenerating dataset: n={n}, vars={vars}, lag={lag}, "
                              f"noise={noise_type}, block_prob={prob:.1f}")

                        # Configure generator
                        noise_params = {'scale': 0.1, 'df': 3} if noise_type == 'student_t' else {'scale': 0.1}
                        generator = BlockMissingNonlinearGenerator(
                            block_size_range=(5, 20),
                            block_prob=prob,
                            noise_type=noise_type,
                            noise_params=noise_params,
                            random_state=42
                        )

                        # Generate dataset
                        df_missing, df_complete, missing_mask = generator.generate_multivariate_ts(
                            n_points=n,
                            n_vars=vars,
                            max_lag=lag,
                            total_time=100,
                            min_gap=0.1
                        )

                        # Save dataset and create visualizations
                        save_dataset_and_visualizations(
                            df_missing=df_missing,
                            df_complete=df_complete,
                            missing_mask=missing_mask,
                            n_vars=vars,
                            max_lag=lag,
                            sample_size=n,
                            noise_type=noise_type,
                            block_prob=prob
                        )
                        print("Dataset and visualizations saved successfully")

Generating nonlinear time series with block missingness...

Generating dataset: n=500, vars=4, lag=2, noise=gaussian, block_prob=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise=gaussian, block_prob=0.2
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise=gaussian, block_prob=0.3
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise=student_t, block_prob=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise=student_t, block_prob=0.2
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise=student_t, block_prob=0.3
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, noise=gaussian, block_prob=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, noise=gaussian, block_prob=0.2
Dataset and visualizatio

/usr/local/lib/python3.11/dist-packages/tigramite/plotting.py:3203: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = pyplot.figure(figsize=figsize)
<ipython-input-4-b0bbbc778a96>:294: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(15, 10))
<ipython-input-4-b0bbbc778a96>:310: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly close

Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=2, noise=gaussian, block_prob=0.3


<ipython-input-4-b0bbbc778a96>:270: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 12))


Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=2, noise=student_t, block_prob=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=2, noise=student_t, block_prob=0.2
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=2, noise=student_t, block_prob=0.3
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=3, noise=gaussian, block_prob=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=3, noise=gaussian, block_prob=0.2
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=3, noise=gaussian, block_prob=0.3
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=3, noise=student_t, block_prob=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=6, lag=3, noise=student_t, block_prob=0.2
Dataset and visualizations saved su

...

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

In [5]:
!zip -r /content/output_D2.zip /content/output_block_missing

  adding: content/output_block_missing/ (stored 0%)
  adding: content/output_block_missing/nonlinear_ts_n1000_vars6_lag3_gaussian_block30_causal_graph.png (deflated 6%)
  adding: content/output_block_missing/nonlinear_ts_n1000_vars8_lag2_gaussian_block30_complete.csv (deflated 54%)
  adding: content/output_block_missing/nonlinear_ts_n5000_vars4_lag4_gaussian_block30_causal_structure.txt (deflated 53%)
  adding: content/output_block_missing/nonlinear_ts_n1000_vars4_lag4_gaussian_block10_statistics.txt (deflated 34%)
  adding: content/output_block_missing/nonlinear_ts_n1000_vars6_lag4_gaussian_block20_series.png (deflated 1%)
  adding: content/output_block_missing/nonlinear_ts_n500_vars8_lag2_student_t_block10_causal_structure.txt (deflated 62%)
  adding: content/output_block_missing/nonlinear_ts_n3000_vars8_lag4_student_t_block30_series.png (deflated 4%)
  adding: content/output_block_missing/nonlinear_ts_n500_vars8_lag3_student_t_block30_missing_pattern.png (deflated 8%)
  adding: cont

In [6]:
from google.colab import files
files.download('/content/output_D2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>